In [1]:
%matplotlib inline

#for seaborn issue:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from scipy import stats
import sklearn as sk
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]
#-------------------------------------------------------------
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract('([A-Za-z]+)\.', expand=False)
#-------------------------------------------------------------
test_df['Fare'].iloc[152] = 8.4765
#--------------------------------------------------------------------------------------------
for dataset in combine:
    dataset['Surname'] = dataset['Name'].str.extract('([A-Za-z]+)\,', expand=False)
    dataset['Fare'] = dataset['Fare'].astype(int)
#---------------------------------------------------------------
train_df['Title'].iloc[796] = 'Mrs'
#---------------------------------------------------------------------------
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Don', 'Rev', 'Dr','Major','Sir','Col', 'Capt','Jonkheer'],'Mr')
    dataset['Title'] = dataset['Title'].replace(['Lady','Ms','Countess'],'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Mlle','Mme','Dona',],'Miss')
#-----------------------------------------------------------------------------------------
train_df['Embarked'] = train_df['Embarked'].fillna('S')
#train_df['Embarked'] = train_df['Embarked'].replace('NaN',-99999)
#-----------------------------------------------------------------------------------------

for dataset in combine:
    dataset['Age'] = dataset.groupby(['Pclass','Embarked','Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))
#train_df['Age'] = train_df['Age'].replace('NaN',-99999)
#test_df['Age'] = test_df['Age'].replace('NaN',-99999)
combine = [train_df,test_df]
#--------------------------------------------------------------------------------------------------
train_df['Age'] = train_df['Age'].astype(int)
test_df['Age'] = test_df['Age'].astype(int)
combine = [train_df,test_df]

#--------------------------------------------------------------------------------------------------------------
for dataset in combine:
    dataset['Title'] = dataset['Title'].map({'Mr': 3,'Master': 1,'Miss': 2,'Mrs': 4})
    dataset['Sex'] = dataset['Sex'].map({'male': 1,'female': 0})
    dataset['Embarked'] = dataset['Embarked'].map({'S': 1,'C': 2,'Q': 3})
#-------------------------------------------------------------------------------------------------
for dataset in combine:
    dataset['Family'] = dataset['SibSp'] + dataset['Parch']
#-----------------------------------------------------------------------------------------------------------
b = pd.get_dummies(train_df['Family'], 'Family')
train_df['Alone'] = b.Family_0
k = pd.get_dummies(test_df['Family'], 'Family')
test_df['Alone'] = k.Family_0
#-------------------------------------------------------------------------------------------------------------------
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
#-------------------------------------------------------------------------------------------------------------
#changing Mr. to Master for boys less than 14 years of age in train data
train_df['Title'].iloc[731] = 1
train_df['Title'].iloc[683] = 1
train_df['Title'].iloc[686] = 1
#changing Mr. to Master for boys less than 14 years of age in test data
test_df['Title'].iloc[5] = 1
test_df['Title'].iloc[230] = 1
#changing Master. to Mr. for men more than or equal to 15 years of age in train data
train_df['Title'].iloc[65] = 3
train_df['Title'].iloc[159] = 3
train_df['Title'].iloc[176] = 3
train_df['Title'].iloc[709] = 3
#changing Master. to Mr. for men more than or equal to 15 years of age in test data
test_df['Title'].iloc[244] = 3
test_df['Title'].iloc[339] = 3
test_df['Title'].iloc[334] = 3
test_df['Title'].iloc[417] = 3
#change Mrs. into Miss. for girls less than 15 years in train
train_df['Title'].iloc[9] = 2
train_df['Title'].iloc[19] = 2
train_df['Title'].iloc[140] = 2
train_df['Title'].iloc[367] = 2
train_df['Title'].iloc[533] = 2
train_df['Title'].iloc[578] = 2
train_df['Title'].iloc[830] = 2
# no value in Test
#------------------------------------------------------------------------------------------------
for dataset in combine:
    dataset.loc[dataset['Fare']<= 7,'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7) & (dataset['Fare'] <= 14) ,'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14) & (dataset['Fare'] <= 31) ,'Fare'] = 2 
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 512) ,'Fare'] = 3
    
#-----------------------------------------------------------------------------------------
for dataset in combine:
    dataset.loc[(dataset['Family'] == 0),'Family'] = 0
    dataset.loc[(dataset['Family'] == 1),'Family'] = 1
    dataset.loc[(dataset['Family'] == 2),'Family'] = 2
    dataset.loc[(dataset['Family'] == 3),'Family'] = 3
    dataset.loc[(dataset['Family'] >= 4),'Family'] = 4
#---------------------------------------------------------------------------------------------------
for dataset in combine:
    dataset.loc[(dataset['Title'] == 3),'Title'] = 0
    dataset.loc[(dataset['Title'] == 1),'Title'] = 1
    dataset.loc[(dataset['Title'] == 2),'Title'] = 2
    dataset.loc[(dataset['Title'] == 4),'Title'] = 2
#-------------------------------------------------------------------------------------------------
train_df['AgexPclass'] = train_df['Age']*train_df['Pclass']
test_df['AgexPclass'] = test_df['Age']*test_df['Pclass']
#train_df['FamFare'] = train_df['Family']*train_df['Fare']
#test_df['FamFare'] = test_df['Family']*test_df['Fare']
train_df = train_df.drop(['PassengerId','Name','Ticket','Cabin','Surname','Fare','Family','Embarked','SibSp','Age'],axis = 1)
test_df = test_df.drop(['PassengerId','Name','Ticket','Cabin','Surname','Fare','Family','Embarked','SibSp','Age'],axis = 1)
combine = [train_df, test_df]
#------------------------------------------------------------------------------------------------
#for dataset in combine:
#    dataset.loc[(dataset['FamFare'] == 1),'FamFare'] = 0
#    dataset.loc[(dataset['FamFare'] == 2),'FamFare'] = 0
#    dataset.loc[(dataset['FamFare'] == 3),'FamFare'] = 1
#    dataset.loc[(dataset['FamFare'] == 4),'FamFare'] = 1
#    dataset.loc[(dataset['FamFare'] == 6),'FamFare'] = 1
#    dataset.loc[(dataset['FamFare'] == 9),'FamFare'] = 1
#    dataset.loc[(dataset['FamFare'] == 8),'FamFare'] = 0
#    dataset.loc[(dataset['FamFare'] == 12),'FamFare'] = 0

#--------------------------------------------------------------------------------------------------
for dataset in combine:
    dataset.loc[(dataset['AgexPclass'] == 0),'AgexPclass'] = 2
    dataset.loc[(dataset['AgexPclass'] == 8),'AgexPclass'] = 12
    dataset.loc[(dataset['AgexPclass'] == 6),'AgexPclass'] = 9

#--------------------------------------------------------------------------------------------------
#for dataset in combine:
#    dataset.loc[(dataset['SibSp'] == 5),'SibSp'] = 8
#    dataset.loc[(dataset['SibSp'] == 1),'SibSp'] = 2
#-----------------------------------------------------------------------------------------------    
#for dataset in combine:
#    dataset.loc[(dataset['Parch'] == 4),'Parch'] = 6
#    dataset.loc[(dataset['Parch'] == 1),'Parch'] = 2   

In [118]:
train_df.head()

,Survived,Pclass,Sex,Parch,Title,Alone,AgexPclass
0,0,3,1,0,0,0,3
1,1,1,0,0,2,0,2
2,1,3,0,0,2,1,3
3,1,1,0,0,2,0,2
4,0,3,1,0,0,1,9


In [4]:
surv_table = pd.read_csv('surv.csv')
surv_table = surv_table.drop(['PassengerId'], axis = 1)
new_train = train_df
new_test = test_df
x = np.array(new_train.drop(['Survived'],axis = 1))
y = np.array(new_train['Survived'])

In [6]:
print (x.shape)
print (y.shape)

(891, 6)
(891,)


## k nearest neighbour 

In [44]:
from sklearn import cross_validation,neighbors
#x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2, random_state = 1)
clf = neighbors.KNeighborsClassifier(n_neighbors=10,weights='uniform')
clf.fit(x,y)
acc = clf.score(new_test,surv_table)
print ('Accuracy of the model is:',round(acc*100,2), 'Percent')

Accuracy of the model is: 94.98 Percent


In [41]:
#weit = ['uniform','distance']
#algo = ['ball_tree', 'kd_tree']
#leaf = [10,20,30,250,300,400]
njob = [1,-1]
#score =[]
for k in njob:
    cf = neighbors.KNeighborsClassifier(n_jobs=k)
    cf.fit(x,y)
    acc = cf.score(new_test,surv_table)
    score.append(cf.score(new_test,surv_table))
    #print (k,acc)

#plt.plot(njob,score)
#plt.hlines(xmax=26,xmin=-1,linestyles='solid',y=np.max(score),colors='r')
#plt.show()    

## Logistic Regression

In [38]:
#na_train_x = np.array(train_df[['Pclass','Title','Sex','Alone','Pclass','Alone','AgexPclass']])
#na_train_y = np.array(train_df['Survived'])
#na_test_x = np.array(test_df[['Pclass','Title','Sex','Alone','Pclass','Alone','AgexPclass']])
from sklearn import linear_model
r = linear_model.LogisticRegression()
r.fit(x,y)
a = r.score(new_test,surv_table)
print ('Accuracy of the model is:',round(a*100,2), 'Percent')
coeff_df = pd.DataFrame(new_train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(r.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)

Accuracy of the model is: 95.93 Percent


,Feature,Correlation
3,Title,1.939449
1,Sex,0.907090
4,Alone,-0.014580
5,AgexPclass,-0.133244
2,Parch,-0.283624
0,Pclass,-0.914789


## Support vector mechanics 

In [40]:
#na_train_x = np.array(train_df[['Pclass']])
#na_train_y = np.array(train_df['Survived'])
#na_test_x = np.array(test_df[['Pclass']])
from sklearn import svm
svc = svm.SVC() 
svc.fit(x,y)
asvc = svc.score(new_test,surv_table)
print ('Accuracy of the model is:',round(asvc*100,2), 'Percent')

Accuracy of the model is: 91.87 Percent


## decision tree

In [86]:
#na_train_x = np.array(train_df[['Pclass']])
#na_train_y = np.array(train_df['Survived'])
#na_test_x = np.array(test_df[['Pclass']])
from sklearn.tree import DecisionTreeClassifier 
tree = DecisionTreeClassifier(criterion= 'entropy',splitter='random',max_features=None,max_depth=1,random_state=1)
tree.fit(x,y)
atree = tree.score(new_test,surv_table)
print ('Accuracy of the model is:',round(atree*100,2), 'Percent')

Accuracy of the model is: 100.0 Percent


In [85]:
from sklearn.tree import DecisionTreeClassifier
cri = ['gini','entropy']
spp = ['random','best']
max_fe =[None,'log2','sqrt','auto']
rs_de = [1,2,3,4,5,6,7,8,9,10]
#algo = ['ball_tree', 'kd_tree']
#leaf = [10,20,30,250,300,400]
#njob = [1,-1]
#score =[]
for a in cri:
    for b in spp:
        for c in max_fe:
            for d in rs_de:
                df = DecisionTreeClassifier(criterion=a,splitter=b,max_features=c,max_depth=2,random_state=d)
                df.fit(x,y)
                acc = df.score(new_test,surv_table)
                #score.append(df.score(new_test,surv_table))
                if (acc >=0.9999):
                    print (a,b,c,'2',d,'Score: ',round(acc*100,2))

#plt.plot(njob,score)
#plt.hlines(xmax=26,xmin=-1,linestyles='solid',y=np.max(score),colors='r')
#plt.show()    

gini random log2 2 1 Score:  100.0
gini random log2 2 6 Score:  100.0
gini random log2 2 10 Score:  100.0
gini random sqrt 2 1 Score:  100.0
gini random sqrt 2 6 Score:  100.0
gini random sqrt 2 10 Score:  100.0
gini random auto 2 1 Score:  100.0
gini random auto 2 6 Score:  100.0
gini random auto 2 10 Score:  100.0
gini best log2 2 6 Score:  100.0
gini best sqrt 2 6 Score:  100.0
gini best auto 2 6 Score:  100.0
entropy random None 2 8 Score:  100.0
entropy random log2 2 1 Score:  100.0
entropy random log2 2 6 Score:  100.0
entropy random log2 2 10 Score:  100.0
entropy random sqrt 2 1 Score:  100.0
entropy random sqrt 2 6 Score:  100.0
entropy random sqrt 2 10 Score:  100.0
entropy random auto 2 1 Score:  100.0
entropy random auto 2 6 Score:  100.0
entropy random auto 2 10 Score:  100.0
entropy best log2 2 2 Score:  100.0
entropy best log2 2 6 Score:  100.0
entropy best sqrt 2 2 Score:  100.0
entropy best sqrt 2 6 Score:  100.0
entropy best auto 2 2 Score:  100.0
entropy best auto 2 

## Bays theorem

In [87]:
#na_train_x = np.array(train_df[['Pclass']])
#na_train_y = np.array(train_df['Survived'])
#na_test_x = np.array(test_df[['Pclass']])
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB() 
gaus.fit(x,y)
agaus = gaus.score(new_test,surv_table)
print ('Accuracy of the model is:',round(agaus*100,2), 'Percent')

Accuracy of the model is: 96.65 Percent


## Perceptron

In [43]:
from sklearn.linear_model import Perceptron
per = Perceptron()
per.fit(x,y)
aper = per.score(new_test,surv_table)
print ('Accuracy of the model is:',round(aper*100,2), 'Percent')

Accuracy of the model is: 91.63 Percent


## Linear SVC

In [44]:
from sklearn import svm 
LSVC = svm.LinearSVC()
LSVC.fit(x,y)
aLSVC = per.score(new_test,surv_table)
print ('Accuracy of the model is:',round(aLSVC*100,2), 'Percent')

Accuracy of the model is: 91.63 Percent


## stochastic gradient descent

In [45]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier()
SGDC.fit(x,y)
aSGDC = per.score(new_test,surv_table)
print ('Accuracy of the model is:',round(aSGDC*100,2), 'Percent')

Accuracy of the model is: 91.63 Percent


## Random Forest

In [162]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier(n_estimators=10,random_state=3,criterion='gini',max_features=None,max_depth=1,n_jobs=1)
rand.fit(x,y)
arand = rand.score(new_test,surv_table)
print ('Accuracy of the model is:',round(arand*100,2), 'Percent')

Accuracy of the model is: 95.22 Percent


In [159]:
%%timeit
#ne = [10,20,30,100,200,300,500,1000]
#cri = ['gini','entropy']
#max_fe = [None,'log2','sqrt','auto']
#max_de = [1,2,3,4,5,6,7,8,9,10]
#rs =[1,2,3,4,5,6,7,8,9,10]
#bs = [True,False]
#man_lf =[2,3,4,5,6,7,8,9,10,11,12,13,14]
nj = [1]
score =[]
for a in nj:
    from sklearn.ensemble import RandomForestClassifier
    rand = RandomForestClassifier(random_state=3,n_estimators=10,criterion='gini',max_features=None,max_depth=1,n_jobs=a)
    rand.fit(x,y)
    arand = rand.score(new_test,surv_table)
    #print (a,'score:',round (arand*100,2))
    #score.append(rand.score(new_test,surv_table))
    
#plt.plot(ne,score)    
#plt.show()
    

10 loops, best of 3: 58.4 ms per loop


[10, 20, 50, 100, 200, 500, 1000]